# Phase 1 

### Preprocessing

In [11]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

### Now load the pickle data into dataframes from the Output folder one directory above

In [22]:
def loadEverything():
    all_projects_bugreports = pd.read_pickle('../Output/allBugReports.pickle')
    print("*** All Bug Reports are Loaded. ***")
    all_projects_source_codes = pd.read_pickle('../Output/allSourceCodes.pickle')
    print("*** All Source Codes are Loaded. ***")
    return all_projects_bugreports, all_projects_source_codes

all_projects_bugreports, all_projects_source_codes = loadEverything()
display(all_projects_bugreports.iloc[0])
display(all_projects_source_codes.iloc[0].unprocessed_code)



*** All Bug Reports are Loaded. ***
*** All Source Codes are Loaded. ***


fix                  [org.apache.commons.collections.map.flat3map.j...
text                                                               NaN
fixdate                                            2006-07-18 22:02:11
summary              Flat3Map.Entry.setValue() overwrites other Ent...
description          Flat3Map&amp;apos;s Entry objects will overwri...
project                                                    COLLECTIONS
average_precision                                                    0
Name: 217, dtype: object

'/*\n *  Licensed to the Apache Software Foundation (ASF) under one or more\n *  contributor license agreements.  See the NOTICE file distributed with\n *  this work for additional information regarding copyright ownership.\n *  The ASF licenses this file to You under the Apache License, Version 2.0\n *  (the "License"); you may not use this file except in compliance with\n *  the License.  You may obtain a copy of the License at\n *\n *      http://www.apache.org/licenses/LICENSE-2.0\n *\n *  Unless required by applicable law or agreed to in writing, software\n *  distributed under the License is distributed on an "AS IS" BASIS,\n *  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n *  See the License for the specific language governing permissions and\n *  limitations under the License.\n */\npackage org.apache.commons.collections;\n\nimport java.util.ArrayList;\nimport java.util.EmptyStackException;\n\n/**\n * An implementation of the {@link java.util.Stack}

## Removing composite varaibles

In [23]:
#remove next line characters:
def remove_new_lines(x):
    return x.replace('\n', '').replace('*', '').replace('/', '')

def clean_new_lines(df):
    df.unprocessed_code = df.unprocessed_code.apply(remove_new_lines)
    return df

df = clean_new_lines(all_projects_source_codes)
df

,filename,unprocessed_code,project
0,\gitrepo\src\java\org\apache\commons\collectio...,Licensed to the Apache Software Foundation ...,COLLECTIONS
1,\gitrepo\src\java\org\apache\commons\collectio...,Licensed to the Apache Software Foundation ...,COLLECTIONS
2,\gitrepo\src\java\org\apache\commons\collectio...,Licensed to the Apache Software Foundation ...,COLLECTIONS
3,\gitrepo\src\java\org\apache\commons\collectio...,Licensed to the Apache Software Foundation ...,COLLECTIONS
4,\gitrepo\src\java\org\apache\commons\collectio...,Licensed to the Apache Software Foundation ...,COLLECTIONS
...,...,...,...
63,\gitrepo\src\main\java\org\wildfly\security\ut...,"JBoss, Home of Professional Open Source. Co...",ELY
64,\gitrepo\src\main\java\org\wildfly\security\_p...,"JBoss, Home of Professional Open Source. Co...",ELY
65,\gitrepo\src\test\java\org\wildfly\security\ma...,"JBoss, Home of Professional Open Source. Co...",ELY
66,\gitrepo\src\test\java\org\wildfly\security\ma...,"JBoss, Home of Professional Open Source. Co...",ELY


In [24]:
import re

#splitting composite words
def findCompositeWords(s):
    return ' '.join(re.findall('[A-Z][^A-Z]*', s))   


def clean_name_column(df):
    df.unprocessed_code = df.unprocessed_code.apply(findCompositeWords)
    return df

df = clean_name_column(all_projects_source_codes)
df

,filename,unprocessed_code,project
0,\gitrepo\src\java\org\apache\commons\collectio...,Licensed to the Apache Software Foundation ...,COLLECTIONS
1,\gitrepo\src\java\org\apache\commons\collectio...,Licensed to the Apache Software Foundation ...,COLLECTIONS
2,\gitrepo\src\java\org\apache\commons\collectio...,Licensed to the Apache Software Foundation ...,COLLECTIONS
3,\gitrepo\src\java\org\apache\commons\collectio...,Licensed to the Apache Software Foundation ...,COLLECTIONS
4,\gitrepo\src\java\org\apache\commons\collectio...,Licensed to the Apache Software Foundation ...,COLLECTIONS
...,...,...,...
63,\gitrepo\src\main\java\org\wildfly\security\ut...,"J Boss, Home of Professional Open Source. ...",ELY
64,\gitrepo\src\main\java\org\wildfly\security\_p...,"J Boss, Home of Professional Open Source. ...",ELY
65,\gitrepo\src\test\java\org\wildfly\security\ma...,"J Boss, Home of Professional Open Source. ...",ELY
66,\gitrepo\src\test\java\org\wildfly\security\ma...,"J Boss, Home of Professional Open Source. ...",ELY
